In [1]:
# to do
# 1. get data of derivatives for analysed subjects
# 2. compute for each participant / each ROI / pos + neg, 
#           - weighted correlation of ecc/size
#           - weigthed laterality index
#           - median r2 fit
#           - mean r2 fit
# 3. compute mean/median/std/sem/95ci across participants for each ROI / pos+neg

# 3. draw for each subject for early-vision pos / mid-vision pos/ DMN ROIs neg (3 rows)
#           - lines weighted correlation size/ecc as with legend as ROI (column 1)
#           - size/ecc correlation r2 as a function of ROIs (column 2)
#           - laterality index as function of ROIs (column 3)
# 4. draw across subjects for early-vision / mid-vision / DMN ROIs (3 rows)
#           - lines weighted correlation size/ecc as with legend as ROI (column 1)
#           - size/ecc correlation r2 as a function of ROIs (column 2)
#           - laterality index as function of ROIs (column 3)
# 5. draw across subjects for early-vision / mid-vision / DMN ROIs (3 columns)
#           - weighted size/ecc correlation as a function ranked subject mean r2 fit in V1 with ROI in legend (row 1)
#           - weighted contra-laterality index as a function ranked subject mean r2 fit in V1 with ROI in legend (row 2)



In [2]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

# # Stop warnings
# # -------------
# import warnings
# warnings.filterwarnings("ignore")

# # General imports
# # ---------------
# import os
# import sys
# import json
# import glob
# import numpy as np
# import ipdb
# import platform
# import h5py
# from scipy.optimize import curve_fit
# from sklearn.metrics import r2_score
# opj = os.path.join
# deb = ipdb.set_trace


In [3]:
# # Define analysis parameters
# # --------------------------
# with open('../../settings.json') as f:
#     json_s = f.read()
#     analysis_info = json.loads(json_s)
    
# # Define cluster/server specific parameters
# # -----------------------------------------
# if 'aeneas' in platform.uname()[1]:
#     base_dir = analysis_info['aeneas_base_folder'] 
# elif 'local' in platform.uname()[1]:
#     base_dir = analysis_info['local_base_folder'] 
    
# sign_idx, rsq_idx, ecc_idx, polar_real_idx, polar_imag_idx , size_idx, \
#             non_lin_idx, amp_idx, baseline_idx, cov_idx, x_idx, y_idx = 0,1,2,3,4,5,6,7,8,9,10,11
    
# fit_model = 'gauss'

In [4]:
# # find analysed subjects
# analysed_subject = []
# for subject in analysis_info['subject_list']:
#     if os.path.isdir(opj(base_dir,'pp_data',subject,fit_model,'h5')) == 1:
#         analysed_subject.append(subject)

# # add to list the time course averaged subjects
# analysed_subject_plus = analysed_subject
# analysed_subject_plus.append('999999')
# analysed_subject_plus.append('999999_ns13')
# analysed_subject_plus.append('999999_mm16')
# analysed_subject_plus.append('999999_mm16_ns13')

In [5]:
# # create all subject folder: 000000
# subject_all = '000000'

# # create h5 file
# summary_hdf5 = opj(base_dir,'pp_data',subject_all,fit_model,'h5')
# try: os.makedirs(summary_hdf5)
# except: pass
# summary_hdf5_file = opj(base_dir,'pp_data',subject_all,fit_model,'h5','summary.h5')
# try: 
#     os.system('rm '+ summary_hdf5_file)
# except: 
#     pass
# h5file = h5py.File(summary_hdf5_file, "a")

In [6]:
# # get analysed subject derivatives
# for subject in analysed_subject_plus:
#     print(subject)
#     for roi_num, roi in enumerate(analysis_info['rois']):
#         for mask_dir in ['pos','neg']:
#             deriv_mat_hemi = []
#             deriv_mat = np.array([])
#             summary_mat = np.array([])
#             val_hemi = 0
#             for hemi in ['L', 'R','LR']:
#                 h5_dir = opj(base_dir,'pp_data',subject,fit_model,'h5')

#                 # load data
#                 if hemi == 'LR':
#                     deriv_mat = np.row_stack((deriv_mat_hemi[0],deriv_mat_hemi[1]))
#                     draw = True
#                 else:
#                     # load derivatives
#                     if hemi == 'L': val_hemi = 1
#                     elif hemi == 'R': val_hemi = 2
#                     folder_alias = '{hemi}_{mask_dir}'.format(hemi = hemi,mask_dir = mask_dir)
#                     h5_file = h5py.File(opj(h5_dir,'{roi}.h5'.format(roi = roi)), "r")
#                     in_file = opj("prf_deriv_{hemi}_{mask_dir}".format(hemi = hemi, mask_dir = mask_dir))
#                     deriv_mat = h5_file['{folder_alias}/{in_file}'.format(folder_alias=folder_alias,in_file=in_file)]
#                     deriv_mat = np.vstack((deriv_mat,val_hemi*np.ones((1,deriv_mat.shape[1]))))
#                     deriv_mat = deriv_mat[:,:].T
#                     deriv_mat_hemi.append(deriv_mat)
            
#             vertex_ini = deriv_mat.shape[0]
#             if vertex_ini > 0:
#                 data4mask = deriv_mat
#                 deriv_mat = deriv_mat[np.logical_and(np.logical_and( data4mask[:,rsq_idx]>=analysis_info['rsq_threshold'],
#                                                                      data4mask[:,cov_idx]>=analysis_info['cov_threshold']),
#                                                                      data4mask[:,size_idx]>=analysis_info['size_threshold'])]
#                 vertex = deriv_mat.shape[0]

#                 if np.round(vertex) == 0:
#                     summary_mat = np.array([np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
#                 else:
#                     # compute correlation size/ecc
#                     if vertex >= 2:
#                         linear_function = lambda x, a, b: a * x + b

#                         ecc_data = deriv_mat[:,ecc_idx]
#                         size_data = deriv_mat[:,size_idx]
#                         weight_data = deriv_mat[:,rsq_idx]
#                         coeffs, matcov = curve_fit( f = linear_function,
#                                                     xdata = ecc_data,
#                                                     ydata = size_data,
#                                                     sigma = weight_data)
#                         size_fit = linear_function(ecc_data, coeffs[0], coeffs[1])

#                         ecc_size_r2 = r2_score(y_true=size_data,y_pred=size_fit)
#                         ecc_size_slope = coeffs[0]
#                         ecc_size_intercept = coeffs[0]
#                     else:
#                         ecc_size_r2 = np.nan
#                         ecc_size_slope = np.nan
#                         ecc_size_intercept = np.nan
                    
#                     # compute laterality index                    
#                     data_L = deriv_mat[deriv_mat[:,12]==1,:]
#                     contra_lat_L = np.nansum(data_L[data_L[:,x_idx] > 0,rsq_idx])/np.nansum(data_L[:,rsq_idx])
#                     data_R = deriv_mat[deriv_mat[:,12]==2,:]
#                     contra_lat_R = np.nansum(data_R[data_R[:,x_idx] < 0,rsq_idx])/np.nansum(data_R[:,rsq_idx])
#                     contra_lat = np.nanmean([contra_lat_L,contra_lat_R])
                    
#                     # median r2 fit
#                     median_fit_r2 = np.nanmedian(deriv_mat[:,rsq_idx])
#                     mean_fit_r2   = np.nanmean(deriv_mat[:,rsq_idx])
                    
#                     summary_mat = np.array([ecc_size_r2,ecc_size_slope,ecc_size_intercept,contra_lat,median_fit_r2,mean_fit_r2])
#             else:
#                 summary_mat = np.array([np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])        

#             # creat subject folder
#             h5file.create_dataset('{subject}/{roi}/{mask_dir}_summary'.format(subject = subject, roi = roi, mask_dir = mask_dir),data = summary_mat,dtype='float32')


In [7]:
# # read file
# h5file = h5py.File(summary_hdf5_file, "a")
# num_sum_val = 6

# # make mean across participants of all parameters
# for roi_num, roi in enumerate(analysis_info['rois']):
#     print(roi)
#     for mask_dir in ['pos','neg']:
#         summary_mat = np.zeros((len(analysed_subject),num_sum_val))
#         for subject_num, subject in enumerate(analysed_subject):
#             folder_alias = '{subject}/{roi}/{mask_dir}_summary'.format(subject = subject, roi = roi, mask_dir = mask_dir)
#             summary_mat[subject_num,:] = h5file['{folder_alias}'.format(folder_alias=folder_alias)]

#         summary_mat_mean = np.nanmean(summary_mat,axis=0)
#         h5file.create_dataset('{subject}/{roi}/{mask_dir}_summary_mean'.format(subject = subject_all, roi = roi, mask_dir = mask_dir),data = summary_mat_mean,dtype='float32')
#         summary_mat_median =  np.nanmedian(summary_mat,axis=0)
#         h5file.create_dataset('{subject}/{roi}/{mask_dir}_summary_median'.format(subject = subject_all, roi = roi, mask_dir = mask_dir),data = summary_mat_median,dtype='float32')
#         summary_mat_std = np.nanstd(summary_mat,axis=0)
#         h5file.create_dataset('{subject}/{roi}/{mask_dir}_summary_std'.format(subject = subject_all, roi = roi, mask_dir = mask_dir),data = summary_mat_std,dtype='float32')
#         summary_mat_sem = scipy.stats.sem(summary_mat,axis=0)
#         h5file.create_dataset('{subject}/{roi}/{mask_dir}_summary_sem'.format(subject = subject_all, roi = roi, mask_dir = mask_dir),data = summary_mat_sem,dtype='float32')
#         summary_mat_95ci = np.nanpercentile(summary_mat,[2.5,97.5],axis=0)
#         h5file.create_dataset('{subject}/{roi}/{mask_dir}_summary_95ci'.format(subject = subject_all, roi = roi, mask_dir = mask_dir),data = summary_mat_95ci,dtype='float32')
        


            

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Stop warnings
# -------------
import warnings
warnings.filterwarnings("ignore")

# General imports
# ---------------
import os
import sys
import json
import glob
import numpy as np
import ipdb
import platform
import h5py
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
opj = os.path.join
deb = ipdb.set_trace

# Bokeh import
# ------------
from bokeh.plotting import figure 
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.models.annotations import Span, Label
from bokeh.layouts import row, column, gridplot
from bokeh.models import BoxZoomTool, BoxSelectTool, Spacer, WheelZoomTool, PanTool, ResetTool
from bokeh.models.glyphs import Text
from bokeh.models.mappers import LinearColorMapper
from bokeh.io import output_notebook, show,save, output_file, export_png, export_svgs
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead
output_notebook()

# Other imports
# -------------
import numpy as np
import cortex
import matplotlib.colors as colors
import scipy as sy
import numpy as np
import ipdb
deb = ipdb.set_trace

# Define analysis parameters
# --------------------------
with open('../../settings.json') as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)
    
# Define cluster/server specific parameters
# -----------------------------------------
if 'aeneas' in platform.uname()[1]:
    base_dir = analysis_info['aeneas_base_folder'] 
elif 'local' in platform.uname()[1]:
    base_dir = analysis_info['local_base_folder'] 

# Define figure parameters
# ------------------------
es_rsq_idx, es_slop_idx, es_intercept_idx, lat_idx, median_rsq_idx, mean_rsq_idx = 0, 1, 2, 3, 4, 5
subject = sys.argv[1]
fit_model = sys.argv[2]
subject_all = '000000'
subject = '0000000'
fit_model = 'gauss'
eb = 'sem'
avg = '_mean' # or '_median'

num_sum_val = 6
if subject == '000000': avg_subject = True
else:avg_subject = False


Loading BokehJS ...

In [5]:
# get data
if avg_subject:
    end_file_avg = avg
else:
    end_file_avg = ''
h5file = h5py.File(opj(base_dir,'pp_data',subject_all,fit_model,'h5','summary.h5'), "r")

for type_fig in ['early_vis','late_vis','dmn']:
    exec('summary_mat_{type_fig} = np.zeros((len(analysis_info["{type_fig}_rois"]),num_sum_val))*np.nan'.format(type_fig=type_fig))
    exec('summary_eb_top_{type_fig} = np.zeros((len(analysis_info["{type_fig}_rois"]),num_sum_val))*np.nan'.format(type_fig=type_fig))
    exec('summary_eb_bot_{type_fig} = np.zeros((len(analysis_info["{type_fig}_rois"]),num_sum_val))*np.nan'.format(type_fig=type_fig))

    if type_fig == 'early_vis':
        mask_dir = 'pos'
    elif type_fig == 'late_vis':
        mask_dir = 'pos'
    elif type_fig == 'dmn':
        mask_dir = 'neg'
    
    for roi_num,roi in enumerate(analysis_info['{type_fig}_rois'.format(type_fig = type_fig)]):
    
        folder_alias = '{subject}/{roi}/{mask_dir}_summary{end_file}'.format(subject = subject, roi = roi, mask_dir = mask_dir, end_file = end_file_avg)

        exec('summary_mat_{type_fig}[roi_num,:] = h5file["{folder_alias}"]'.format(type_fig = type_fig,folder_alias=folder_alias))
        
        if avg_subject:
            folder_alias = '{subject}/{roi}/{mask_dir}_summary{end_file}'.format(subject = subject, roi = roi, mask_dir = mask_dir, end_file = '_{eb}'.format(eb=eb))
            if eb == '95ci':
                exec('summary_eb_{type_fig} = h5file["{folder_alias}"]'.format(type_fig = type_fig,folder_alias=folder_alias))
                exec('summary_eb_bot_{type_fig}[roi_num,:] = summary_eb_{type_fig}[0,:]'.format(type_fig = type_fig))
                exec('summary_eb_top_{type_fig}[roi_num,:] = summary_eb_{type_fig}[1,:]'.format(type_fig = type_fig))
            else:
                exec('summary_eb_{type_fig} = h5file["{folder_alias}"]'.format(type_fig = type_fig,folder_alias=folder_alias))
                exec('summary_eb_bot_{type_fig}[roi_num,:] = summary_mat_{type_fig}[roi_num,:] - summary_eb_{type_fig}'.format(type_fig = type_fig))
                exec('summary_eb_top_{type_fig}[roi_num,:] = summary_mat_{type_fig}[roi_num,:] + summary_eb_{type_fig}'.format(type_fig = type_fig))


KeyError: 'Unable to open object (component not found)'

In [210]:
# ecc/size figure
# ---------------
p_width = 400
p_height = 400
x_range_es = (0,10)
y_range_es = (0,10)
x_tick_steps_es = 2
y_tick_steps_es = 2
min_border_large = 10
stim_radius = analysis_info['stim_radius']
stim_color = tuple([250,250,250])
bg_color = tuple([229,229,229])
x_label_es = 'Eccentricity (dva)'
y_label_es = 'Size (dva)'
cmap = 'J4'

# define colors
cmap_steps = len(analysis_info['rois'])+1
col_offset = 1/14.0
base = cortex.utils.get_cmap(cmap)
val = np.fmod(np.linspace(0+col_offset, 1+col_offset,cmap_steps,endpoint=False),1.0)
colmap = colors.LinearSegmentedColormap.from_list('my_colmap',base(val),N = cmap_steps)
vmin, vmax = 0, 2*np.pi
data_col = np.linspace(vmin,vmax,cmap_steps)
vrange = float(vmax) - float(vmin)
norm_data = ((data_col-float(vmin))/vrange)*cmap_steps
col_mat_rgb = colmap(norm_data.astype(int)) * 255.0
colors_val_rgb = ["#%02x%02x%02x" % (int(r), int(g), int(b)) for r, g, b in zip(col_mat_rgb[:,0], col_mat_rgb[:,1], col_mat_rgb[:,2])]
early_vis_num = len(analysis_info['early_vis_rois'])
late_vis_num = len(analysis_info['late_vis_rois'])
dmn_num = len(analysis_info['dmn_rois'])
col_early_vis = colors_val_rgb[0:early_vis_num]
col_late_vis = colors_val_rgb[early_vis_num:early_vis_num+late_vis_num]
col_dmn = colors_val_rgb[early_vis_num+late_vis_num:early_vis_num+late_vis_num+dmn_num]

# linear function
linear_function = lambda x, a, b: a * x + b

# define figure
es_fig_all = []
for type_fig in ['early_vis','late_vis','dmn']:
    if type_fig == 'early_vis':
        title = '{sub}: early vision ROIs: pRF sign [+]'.format(sub = subject)
    elif type_fig == 'late_vis':
        title = '{sub}: late vision ROIs: pRF sign [+]'.format(sub = subject)
    elif type_fig == 'dmn':
        title = '{sub}: Default Mode Network ROIs: pRF sign [–]'.format(sub = subject)
    exec('col = col_{type_fig}'.format(type_fig = type_fig))
    list_roi = analysis_info['{type_fig}_rois'.format(type_fig = type_fig)]
        
    es_fig = figure(plot_width = p_width,
                plot_height = p_height,
                x_range = x_range_es,
                y_range = y_range_es,
                min_border_left = min_border_large,
                min_border_right = min_border_large,
                min_border_bottom = min_border_large,
                min_border_top = min_border_large,
                toolbar_location = None,
                title = title,
                tools = "")
    es_fig.xaxis.axis_label = x_label_es
    es_fig.yaxis.axis_label = y_label_es
    es_fig.grid.grid_line_color = None
    es_fig.axis.minor_tick_in = False
    es_fig.axis.minor_tick_out = False
    es_fig.axis.major_tick_in = False
    es_fig.outline_line_alpha = 0
    es_fig.xaxis.ticker = np.arange(x_range_es[0],x_range_es[1] + x_tick_steps_es, x_tick_steps_es)
    es_fig.yaxis.ticker = np.arange(y_range_es[0],y_range_es[1] + y_tick_steps_es, y_tick_steps_es)
    es_fig.background_fill_color = bg_color
    es_fig.axis.axis_label_standoff = 10
    es_fig.axis.axis_label_text_font_style = 'normal'

    # plot stimulus area
    es_fig.quad(bottom = y_range_es[0], left = 0, right = stim_radius, top = y_range_es[1], color = stim_color)

    # plot ecc/size lines
    x_es = np.array((x_range_es[0],x_range_es[1]))
    
    for roi_num,roi in enumerate(list_roi):

        exec('summary_mat = summary_mat_{type_fig}'.format(type_fig = type_fig))
        
        if avg_subject:
            
            exec('summary_eb_bot = summary_eb_bot_{type_fig}'.format(type_fig = type_fig))
            exec('summary_eb_top = summary_eb_top_{type_fig}'.format(type_fig = type_fig))
            

        # get values
        rsq_es = summary_mat[roi_num,es_rsq_idx]
        slope_es = summary_mat[roi_num,es_slop_idx]
        intercept_es = summary_mat[roi_num,es_intercept_idx]
        y_es = linear_function(x_es,slope_es,intercept_es)

        # draw error bar areas
        if avg_subject:
            # bottom
            rsq_es_bot = summary_eb_bot[roi_num,es_rsq_idx]
            slope_es_bot = summary_eb_bot[roi_num,es_slop_idx]
            intercept_es_bot = summary_eb_bot[roi_num,es_intercept_idx]
            y_es_bot = linear_function(x_es,slope_es_bot,intercept_es_bot)

            # top
            rsq_es_top = summary_eb_top[roi_num,es_rsq_idx]
            slope_es_top = summary_eb_top[roi_num,es_slop_idx]
            intercept_es_top = summary_eb_top[roi_num,es_intercept_idx]
            y_es_top = linear_function(x_es,slope_es_top,intercept_es_top)

            # draw error bar
            x_eb = [x_es[0],x_es[1],x_es[1],x_es[0]]
            y_eb = [y_es_bot[0],y_es_bot[1],y_es_top[1],y_es_top[0]]

            if eb == '95ci':
                legend_txt = '{roi} (R\u00b2: {:1.2g} ± [{:1.2g},{:1.2g}])'.format(rsq_es,rsq_es_bot,rsq_es_top,roi = roi)
            else:
                rsq_eb = rsq_es_top - rsq_es
                legend_txt = '{roi} (R\u00b2: {:1.2g} ± {:1.2g})'.format(rsq_es, rsq_eb, roi = roi)
            es_fig.patch(x_eb,y_eb,line_color = None, fill_color = col[roi_num], fill_alpha = 0.1,legend=legend_txt)
        else:
            legend_txt = '{roi} (R\u00b2: {:1.2g})'.format(rsq_es, roi = roi)
            
        # draw line
        es_fig.line(x_es,y_es,line_color=col[roi_num],legend=legend_txt,line_width = 2)
        
        # legend
        es_fig.legend.location = "top_left"
        es_fig.legend.click_policy="hide"
        es_fig.legend.background_fill_alpha = 0
        es_fig.legend.label_text_font = '8pt'
        es_fig.legend.margin = 10
        es_fig.legend.spacing = -2
        es_fig.legend.glyph_width = 10
        es_fig.legend.label_text_baseline = 'middle'
        es_fig.legend.border_line_color = None

    es_fig_all.append(es_fig)

f1 = column( row(es_fig_all[0],es_fig_all[1],es_fig_all[2]))

  
# show(f1)

In [211]:
# ecc/size r2 vs. ROI figures
# ---------------------------
es_roi_fig_all = []
for type_fig in ['early_vis','late_vis','dmn']:
    p_width = 400
    p_height = 400
    x_range_es_roi = (-1,len(analysis_info["{type_fig}_rois".format(type_fig = type_fig)]))
    y_range_es_roi = (-0.5,1)
    x_tick_steps_es_roi = 1
    y_tick_steps_es_roi = 0.25
    x_label_es_roi = 'ROI'
    y_label_es_roi = 'Ecc. vs. Size R\u00b2'
    
    exec('col = col_{type_fig}'.format(type_fig = type_fig))
    
    if type_fig == 'early_vis':
        title = '{sub}: early vision ROIs: pRF sign [+]'.format(sub = subject)
    elif type_fig == 'late_vis':
        title = '{sub}: late vision ROIs: pRF sign [+]'.format(sub = subject)
    elif type_fig == 'dmn':
        title = '{sub}: Default Mode Network ROIs: pRF sign [–]'.format(sub = subject)
    
    list_roi = analysis_info['{type_fig}_rois'.format(type_fig = type_fig)]
        
    es_roi_fig = figure(plot_width = p_width,
                    plot_height = int(p_height*0.4),
                    x_range = x_range_es_roi,
                    y_range = y_range_es_roi,
                    min_border_left = min_border_large,
                    min_border_right = min_border_large,
                    min_border_bottom = min_border_large,
                    min_border_top = min_border_large,
                    toolbar_location = None,
                    tools = "")
    es_roi_fig.xaxis.axis_label = ''
    es_roi_fig.yaxis.axis_label = y_label_es_roi
    es_roi_fig.grid.grid_line_color = None
    es_roi_fig.axis.minor_tick_in = False
    es_roi_fig.axis.minor_tick_out = False
    es_roi_fig.axis.major_tick_in = False
    es_roi_fig.outline_line_alpha = 0
    es_roi_fig.yaxis.ticker = np.arange(y_range_es_roi[0],y_range_es_roi[1] + y_tick_steps_es_roi, y_tick_steps_es_roi)
    es_roi_fig.xaxis.major_label_orientation = np.pi/3
    dict_label = {-1:''}
    for roi_num, roi in enumerate(list_roi):
        dict_label.update({roi_num: roi})
    dict_label.update({roi_num+1:''})
    es_roi_fig.xaxis.major_label_overrides = dict_label

    es_roi_fig.background_fill_color = bg_color
    es_roi_fig.yaxis.axis_label_standoff = 10
    es_roi_fig.xaxis.axis_label_standoff = 10
    es_roi_fig.axis.axis_label_text_font_style = 'normal'
    es_roi_fig.axis.axis_label_text_align = 'left'
    es_roi_fig.xaxis.major_label_text_font_size = '0pt'
        
    # span
    es_roi_fig.add_layout(Span(location = 0, dimension = 'width', line_alpha = 0.5, line_color = 'black', line_width = 1, line_dash = 'dashed'))

    # draw error bar
    if avg_subject:
        exec('summary_eb_bot = summary_eb_bot_{type_fig}'.format(type_fig = type_fig))
        exec('summary_eb_top = summary_eb_top_{type_fig}'.format(type_fig = type_fig))

        x_mat = []
        y_mat = []
        for roi_num,roi in enumerate(list_roi):
            x_mat.append([roi_num,roi_num])
            y_mat.append([summary_eb_bot[roi_num,es_rsq_idx],summary_eb_top[roi_num,es_rsq_idx]])

        es_roi_fig.multi_line(x_mat, y_mat,line_color = col,line_width = 2)
    
    # draw dots
    x_circle = np.zeros(len(list_roi))
    y_circle = np.zeros(len(list_roi))
    for roi_num,roi in enumerate(list_roi):
        exec('summary_mat = summary_mat_{type_fig}'.format(type_fig = type_fig))
        exec('col = col_{type_fig}'.format(type_fig = type_fig))
        x_circle[roi_num] = roi_num
        y_circle[roi_num] = summary_mat[roi_num,es_rsq_idx]

    es_roi_fig.circle(x = x_circle,y = y_circle, size = 10, fill_color = 'white', line_color = col,line_width = 2)

    
    es_roi_fig_all.append(es_roi_fig)
    
f2 = column( row(es_roi_fig_all[0],es_roi_fig_all[1],es_roi_fig_all[2]))

# show(f2)

In [212]:
# Contra-laterality vs. ROI figures
# ---------------------------------
lat_roi_fig_all = []
for type_fig in ['early_vis','late_vis','dmn']:
    p_width = 400
    p_height = 400
    x_range_lat_roi = (-1,len(analysis_info["{type_fig}_rois".format(type_fig = type_fig)]))
    y_range_lat_roi = (0,1)
    x_tick_steps_lat_roi = 1
    y_tick_steps_lat_roi = 0.25
    x_label_lat_roi = 'ROI'
    y_label_lat_roi = 'Contra-lat. index (%)'
    
    exec('col = col_{type_fig}'.format(type_fig = type_fig))
    
    if type_fig == 'early_vis':
        title = '{sub}: early vision ROIs: pRF sign [+]'.format(sub = subject)
    elif type_fig == 'late_vis':
        title = '{sub}: late vision ROIs: pRF sign [+]'.format(sub = subject)
    elif type_fig == 'dmn':
        title = '{sub}: Default Mode Network ROIs: pRF sign [–]'.format(sub = subject)
    
    list_roi = analysis_info['{type_fig}_rois'.format(type_fig = type_fig)]
        
    lat_roi_fig = figure(   plot_width = p_width,
                            plot_height = int(p_height*0.65),
                            x_range = x_range_lat_roi,
                            y_range = y_range_lat_roi,
                            min_border_left = min_border_large,
                            min_border_right = min_border_large,
                            min_border_bottom = min_border_large,
                            min_border_top = min_border_large,
                            toolbar_location = None,
                            tools = "")
    
    lat_roi_fig.xaxis.axis_label = x_label_lat_roi
    lat_roi_fig.yaxis.axis_label = y_label_lat_roi
    lat_roi_fig.grid.grid_line_color = None
    lat_roi_fig.axis.minor_tick_in = False
    lat_roi_fig.axis.minor_tick_out = False
    lat_roi_fig.axis.major_tick_in = False
    lat_roi_fig.outline_line_alpha = 0
    lat_roi_fig.yaxis.ticker = np.arange(y_range_lat_roi[0],y_range_lat_roi[1] + y_tick_steps_lat_roi, y_tick_steps_lat_roi)
    lat_roi_fig.xaxis.major_label_orientation = np.pi/3
    dict_label = {-1:''}
    for roi_num, roi in enumerate(list_roi):
        dict_label.update({roi_num: roi})
    dict_label.update({roi_num+1:''})
    lat_roi_fig.xaxis.major_label_overrides = dict_label

    lat_roi_fig.background_fill_color = bg_color
    lat_roi_fig.yaxis.axis_label_standoff = 10
    lat_roi_fig.xaxis.axis_label_standoff = 10
    lat_roi_fig.axis.axis_label_text_font_style = 'normal'
    lat_roi_fig.axis.axis_label_text_align = 'left'
            
    # draw error bar
    if avg_subject:
        exec('summary_eb_bot = summary_eb_bot_{type_fig}'.format(type_fig = type_fig))
        exec('summary_eb_top = summary_eb_top_{type_fig}'.format(type_fig = type_fig))

        x_mat = []
        y_mat = []
        for roi_num,roi in enumerate(list_roi):
            x_mat.append([roi_num,roi_num])
            y_mat.append([summary_eb_bot[roi_num,lat_idx],summary_eb_top[roi_num,lat_idx]])

        lat_roi_fig.multi_line(x_mat, y_mat,line_color = col,line_width = 2)
    
    # draw dots
    x_circle = np.zeros(len(list_roi))
    y_circle = np.zeros(len(list_roi))
    for roi_num,roi in enumerate(list_roi):
        exec('summary_mat = summary_mat_{type_fig}'.format(type_fig = type_fig))
        exec('col = col_{type_fig}'.format(type_fig = type_fig))
        x_circle[roi_num] = roi_num
        y_circle[roi_num] = summary_mat[roi_num,lat_idx]

    lat_roi_fig.circle(x = x_circle,y = y_circle, size = 10, fill_color = 'white', line_color = col,line_width = 2)

    
    lat_roi_fig_all.append(lat_roi_fig)
    
f3 = column( row(lat_roi_fig_all[0],lat_roi_fig_all[1],lat_roi_fig_all[2]))

# show(f3)

In [213]:
all_f = gridplot([[f1],[f2],[f3]],toolbar_location = None)
show(all_f)

In [207]:
show(all_f)